# softmax与对数似然函数及它俩在PyTorch中的实现细节
__filename__ = "softmax_and_logliklihood_with_pytorch.ipynb"

__date__ = 2019 / 4 / 10

__author__ = "leichao"

__email__ = "leichaocn@163.com"

---
对于一个尺寸为(N,C)的模型输出数据，每一行表示一个样本，共N个样本；每一个样本有C列，表示一个样本分别对应的C个类的预测数值。
对每一行样本，它有C个$z_i$值，我们对于每个$z_i$求取它的softmax值。
经典的softmax为（对于一条样本的输出各维度）
$$ a_{i}=\frac{e^{z_{i}}}{\sum{e^{z_{i}}}}
$$
经典的对数似然函数为（对于N条样本，每条样本仅拿出$y_i=1$对应的那个$a_i$）
$$ loss=-\sum{y_ilog(a_{i})}
$$
其中$y_i$是我们的预测值。$loss$是N条样本上的总损失。

---
在pytorch中有两种实现方式：

**第一种：log_softmax()与nll_loss()组合**

F.log_softmax()干的事是
$$
lsm_i=log(a_i)=log(\frac{e^{z_{i}}}{\sum{e^{z_{i}}}})=z_i-log(\sum{e^{z_{i}}})
$$
F.nll_loss()干的事是
$$ loss=-\sum{y_ilsm_i}
$$

输入到log_softmax中的tensor的尺寸必须是NxC，
每一行表示一个样本，共N个样本；每一个样本有C列，表示一个样本分别对应的C个类的预测数值。
对每一行样本，它有C个$z_i$值，我们对于每个$z_i$求取它的softmax值。

在代码`F.nll_loss(F.log_softmax(input),target)`中
    - input的尺寸为（N，C），
    - input的每一行表示一个样本，共N个样本；每一个样本有C列，表示一个样本分别对应的C个类的预测数值。
    - input的每一行样本，它有C个$z_i$ 值，我们对于每个$z_i$求取它的softmax值。
    - F.log_softmax(input)的输出尺寸为依然为（N，C）
    - target的尺寸为N的一维张量，元素最小为0，最大为C-1
    - nll_loss()的参数，如果reduction='sum'，则loss为全部N个样本的总损失；如果缺省，则loss为全部N个样本的总损失除以N。


**第2种：cross_entropy()**

`F.cross_entropy(input,target)`相当于把softmax与对数似然函数一起做了，它的功能等于`F.nll_loss(F.log_softmax(input),target)`

对比代码见下：


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F


# input假设为模型softmax之前的输出结果，尺寸为（N，C），即N个样本分别对应C个类的预测结果。
input = torch.randn(3, 2, requires_grad=True)
print('input.size()=',input.size())
print('input.item()=',input)


# target中有N个元素，元素的值的范围为[0,C-1]
target = torch.tensor([0, 0, 0])
print('target.size()=',target.size())
print('target.item()=',target)


softmax_input=F.log_softmax(input)
print('softmax_input.size()=',softmax_input.size())
print('softmax_input.item()=',softmax_input)

nll_loss_out = F.nll_loss(softmax_input,target)
# nll_loss_out = F.nll_loss(softmax_input,target,reduction='sum')

crossentropy_loss_out = F.cross_entropy(input,target)
# crossentropy_loss_out = F.cross_entropy(input,target,reduction='sum')

print('nll_loss_out.size()=',nll_loss_out.size())
print('nll_loss_out=',nll_loss_out.item())

print('crossentropy_loss_out.size()=',crossentropy_loss_out.size())
print('crossentropy_loss_out=',crossentropy_loss_out.item())


input.size()= torch.Size([3, 2])
input.item()= tensor([[-0.3641, -0.6573],
        [ 2.0380,  0.0544],
        [-0.5526,  1.8896]], requires_grad=True)
target.size()= torch.Size([3])
target.item()= tensor([0, 0, 0])


D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


softmax_input.size()= torch.Size([3, 2])
softmax_input.item()= tensor([[-0.5573, -0.8504],
        [-0.1289, -2.1125],
        [-2.5257, -0.0834]], grad_fn=<LogSoftmaxBackward>)
nll_loss_out.size()= torch.Size([])
nll_loss_out= 1.0706039667129517
crossentropy_loss_out.size()= torch.Size([])
crossentropy_loss_out= 1.0706039667129517
